In [1]:
! pip3 install -r requirements.txt

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [73]:
import glob
import os
import boto3
import socket
from botocore.handlers import disable_signing
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import time
import warnings
warnings.filterwarnings("ignore")

In [4]:
if socket.gethostname() == 'Rohits-MBP':
    rootdir = '/Users/rohitchanne/Documents/capstone/data/data_parquet/'
else: 
    rootdir = '' # Enter your hone dir here

In [5]:
list_of_files = filter( os.path.isfile, glob.glob(rootdir + '*') )
files_with_size = [ file_path for file_path in list_of_files ]

In [6]:
dfs_parquet = {}
for file_path in files_with_size:
    if 'parquet' in file_path:
        file_name = file_path.split('/')[-1]
        df_name = file_name.split('_')[0]
        print(f'Reading Data File: {file_name}')       
        dfs_parquet[df_name] = pd.read_parquet(file_path, engine='pyarrow')

Reading Data File: Wednesday-21-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-23-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thuesday-20-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-22-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-16-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-28-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-14-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-15-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-01-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-02-03-2018_TrafficForML_CICFlowMeter_clean.parquet


In [25]:
dfs_parquet['Thuesday-20-02-2018'].drop(['Flow ID', 'Src IP', 'Src Port', 'Dst IP'], axis=1, inplace=True)

In [7]:
for k,df in dfs_parquet.items():
    print(k)
    df['is_allowed'] = df['Label'] == 'Benign'
    del df['Label']
    print(f'DF:{k}, Shape{df.shape}')

Wednesday-21-02-2018
DF:Wednesday-21-02-2018, Shape(1048575, 80)
Friday-23-02-2018
DF:Friday-23-02-2018, Shape(1048575, 80)
Thuesday-20-02-2018
DF:Thuesday-20-02-2018, Shape(7948748, 84)
Thursday-22-02-2018
DF:Thursday-22-02-2018, Shape(1048575, 80)
Friday-16-02-2018
DF:Friday-16-02-2018, Shape(1048574, 80)
Wednesday-28-02-2018
DF:Wednesday-28-02-2018, Shape(613071, 80)
Wednesday-14-02-2018
DF:Wednesday-14-02-2018, Shape(1048575, 80)
Thursday-15-02-2018
DF:Thursday-15-02-2018, Shape(1048575, 80)
Thursday-01-03-2018
DF:Thursday-01-03-2018, Shape(331100, 80)
Friday-02-03-2018
DF:Friday-02-03-2018, Shape(1048575, 80)


In [9]:
cat_cols = ['Dst Port']

In [59]:
df_train = pd.concat([dfs_parquet['Friday-02-03-2018'], 
                      dfs_parquet['Friday-16-02-2018'], 
                      dfs_parquet['Friday-23-02-2018'],
                      dfs_parquet['Thursday-01-03-2018'],
                      dfs_parquet['Thursday-15-02-2018'],
                      dfs_parquet['Thursday-22-02-2018'],
#                       dfs_parquet['Thuesday-20-02-2018'],
                      dfs_parquet['Wednesday-14-02-2018'],
                      dfs_parquet['Wednesday-21-02-2018']
                     ]
                    )
#Transforming timestamp and category columns and also scaling data using minmax scalar
df_train['Timestamp'] = pd.to_datetime(df_train['Timestamp'])
df_train['Date'] = pd.to_datetime(df_train['Timestamp']).dt.date
df_train['TS_relative'] = (df_train['Timestamp'].astype(int) - 
                             pd.to_datetime(df_train['Date']).astype(int))/ 10**9
df_train = df_train.drop(['Timestamp'], axis = 1)
df_train = df_train.drop(['Date'], axis = 1)
df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train.fillna(0, inplace=True)
df_train[cat_cols] = df_train[cat_cols].astype('category')
X_train = df_train.drop(['is_allowed'], axis = 1)
y_train = df_train['is_allowed']*1 

In [28]:
stdsc = MinMaxScaler()
stdsc.fit(X_train)
X_train_scaled = stdsc.transform(X_train)

In [54]:
iterations = 50

In [29]:
# df_test = pd.concat([dfs_parquet['Wednesday-14-02-2018'],
#                     dfs_parquet['Wednesday-21-02-2018'],
#                     dfs_parquet['Wednesday-28-02-2018']]
#                      )
df_test = dfs_parquet['Wednesday-28-02-2018']
#Transforming timestamp and category columns and also scaling data using minmax scalar
df_test['Timestamp'] = pd.to_datetime(df_test['Timestamp'])
df_test['Date'] = pd.to_datetime(df_test['Timestamp']).dt.date
df_test['TS_relative'] = (df_test['Timestamp'].astype(int) - 
                             pd.to_datetime(df_test['Date']).astype(int))/ 10**9
df_test = df_test.drop(['Timestamp'], axis = 1)
df_test = df_test.drop(['Date'], axis = 1)
df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
df_test.fillna(0, inplace=True)
df_test[cat_cols] = df_test[cat_cols].astype('category')
X_test = df_test.drop(['is_allowed'], axis = 1)
y_test = df_test['is_allowed'].astype(int)*1

In [30]:
stdsc = MinMaxScaler()
stdsc.fit(X_test)
X_test_scaled = stdsc.transform(X_test)

### Model Training Bayes

In [31]:
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=79, out_features=128),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=128, out_features=2),
)

ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01

optimizer = optim.Adam(model.parameters(), lr=0.01)

In [32]:
x_for_nn =  torch.from_numpy(X_train_scaled).float()
x_for_nn.shape

torch.Size([15619872, 79])

In [33]:
y_for_nn = torch.from_numpy(y_train.to_numpy())
y_for_nn.shape

torch.Size([15619872])

In [34]:
x_for_nn_test =  torch.from_numpy(X_test_scaled).float()
x_for_nn_test.shape

torch.Size([613071, 79])

In [35]:
y_for_nn_test = torch.from_numpy(y_test.to_numpy())
y_for_nn_test.shape

torch.Size([613071])

In [37]:
kl_weight = 0.1
for step in range(100):
    pre = model(x_for_nn)
    ce = ce_loss(pre, y_for_nn)
    kl = kl_loss(model)
    cost = ce + kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if step%10 == 0:
        print(f'Epoch: {step}/100.............')

    
_, predicted = torch.max(pre.data, 1)
print(f'F1: {f1_score(y_for_nn, predicted)}')
print(f'Accuracy: {accuracy_score(y_for_nn, predicted)}')
print(f'Precision: {precision_score(y_for_nn, predicted)}')
print(f'Recall: {recall_score(y_for_nn, predicted)}')

Epoch: 0/100.............
Epoch: 10/100.............
Epoch: 20/100.............
Epoch: 30/100.............
Epoch: 40/100.............
Epoch: 50/100.............
Epoch: 60/100.............
Epoch: 70/100.............
Epoch: 80/100.............
Epoch: 90/100.............
F1: 0.9277979108427403
Accuracy: 0.8867356915600845
Precision: 0.9830814546721328
Recall: 0.878401064316795


### Model Testing Bayes

In [55]:
for i in range(iterations):
    pre_test = model(x_for_nn_test)
    

_, predicted_test = torch.max(pre_test.data, 1)

In [56]:
print(f'F1: {f1_score(y_for_nn_test, predicted_test)}')
print(f'Accuracy: {accuracy_score(y_for_nn_test, predicted_test)}')
print(f'Precision: {precision_score(y_for_nn_test, predicted_test)}')
print(f'Recall: {recall_score(y_for_nn_test, predicted_test)}')

F1: 0.8626468822694336
Accuracy: 0.7637484076069493
Precision: 0.8912994487567142
Recall: 0.835779125321573


### Trying out training with batches

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=1234)

In [61]:
stdsc = MinMaxScaler()
stdsc.fit(X_train)
X_train_scaled = stdsc.transform(X_train)

In [63]:
stdsc = MinMaxScaler()
stdsc.fit(X_test)
X_test_scaled = stdsc.transform(X_test)

In [83]:
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=79, out_features=128),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=128, out_features=2),
)

ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01

optimizer = optim.Adam(model.parameters(), lr=0.01)

In [64]:
x_for_nn =  torch.from_numpy(X_train_scaled).float()
x_for_nn.shape

torch.Size([6136899, 79])

In [65]:
y_for_nn = torch.from_numpy(y_train.to_numpy())
y_for_nn.shape

torch.Size([6136899])

In [66]:
x_for_nn_test =  torch.from_numpy(X_test_scaled).float()
x_for_nn_test.shape

torch.Size([1534225, 79])

In [67]:
y_for_nn_test = torch.from_numpy(y_test.to_numpy())
y_for_nn_test.shape

torch.Size([1534225])

In [68]:
kl_weight = 0.1
for step in range(10):
    pre = model(x_for_nn)
    ce = ce_loss(pre, y_for_nn)
    kl = kl_loss(model)
    cost = ce + kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if step%10 == 0:
        print(f'Epoch: {step}/100.............')

    
_, predicted = torch.max(pre.data, 1)
print(f'F1: {f1_score(y_for_nn, predicted)}')
print(f'Accuracy: {accuracy_score(y_for_nn, predicted)}')
print(f'Precision: {precision_score(y_for_nn, predicted)}')
print(f'Recall: {recall_score(y_for_nn, predicted)}')

Epoch: 0/100.............
F1: 0.8410199569940885
Accuracy: 0.7256552535735068
Precision: 0.7256552535735068
Recall: 1.0


In [69]:
pre_test = model(x_for_nn_test)
_, predicted_test = torch.max(pre_test.data, 1)
print(f'F1: {f1_score(y_for_nn_test, predicted_test)}')
print(f'Accuracy: {accuracy_score(y_for_nn_test, predicted_test)}')
print(f'Precision: {precision_score(y_for_nn_test, predicted_test)}')
print(f'Recall: {recall_score(y_for_nn_test, predicted_test)}')

F1: 0.8416148118674033
Accuracy: 0.7265414134171976
Precision: 0.7265414134171976
Recall: 1.0


In [74]:
batch_size = 32768
train_data = TensorDataset(x_for_nn, y_for_nn)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [78]:
kl_weight = 0.1
epochs = 2
for epoch in range(epochs):
    start_time = time.process_time()
    avg_loss = 0.
    counter = 0
    for x, y in train_loader:
        counter += 1
        pre = model(x)
        ce = ce_loss(pre, y)
        kl = kl_loss(model)
        cost = ce + kl_weight*kl

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        avg_loss += cost.item()
        print(f"Epoch {epoch}...... Average Loss for Epoch: {avg_loss/counter}")
        _, predicted = torch.max(pre.data, 1)
        print(f'F1: {f1_score(y, predicted)}')
        print(f'Accuracy: {accuracy_score(y, predicted)}')
        print(f'Precision: {precision_score(y, predicted)}')
        print(f'Recall: {recall_score(y, predicted)}')            
    current_time = time.process_time()

Epoch 0...... Average Loss for Epoch: 0.589142918586731
F1: 0.8399476050554041
Accuracy: 0.72406005859375
Precision: 0.72406005859375
Recall: 1.0
Epoch 0...... Average Loss for Epoch: 0.5970736145973206
F1: 0.840276057334985
Accuracy: 0.72454833984375
Precision: 0.72454833984375
Recall: 1.0
Epoch 0...... Average Loss for Epoch: 0.6696451902389526
F1: 0.8429787083789414
Accuracy: 0.72857666015625
Precision: 0.72857666015625
Recall: 1.0
Epoch 0...... Average Loss for Epoch: 0.6517206281423569
F1: 0.8384260900389933
Accuracy: 0.7218017578125
Precision: 0.7218017578125
Recall: 1.0
Epoch 0...... Average Loss for Epoch: 0.6426032304763794
F1: 0.8419973848817895
Accuracy: 0.72711181640625
Precision: 0.72711181640625
Recall: 1.0
Epoch 0...... Average Loss for Epoch: 0.6312525471051534
F1: 0.8404607300199933
Accuracy: 0.724822998046875
Precision: 0.724822998046875
Recall: 1.0
Epoch 0...... Average Loss for Epoch: 0.6250875847680228
F1: 0.8396189666772903
Accuracy: 0.72357177734375
Precision: 0.

Epoch 0...... Average Loss for Epoch: 0.5111335935416045
F1: 0.897427646646855
Accuracy: 0.834625244140625
Precision: 0.8187187014332585
Recall: 0.99287987937678
Epoch 0...... Average Loss for Epoch: 0.5072023196653886
F1: 0.9286429286429286
Accuracy: 0.893402099609375
Precision: 0.8986280789151149
Recall: 0.9607320990785359
Epoch 0...... Average Loss for Epoch: 0.503231051244906
F1: 0.9376294980760286
Accuracy: 0.90997314453125
Precision: 0.9438556165666369
Recall: 0.9314849821466079
Epoch 0...... Average Loss for Epoch: 0.49947657292349296
F1: 0.9368776198752684
Accuracy: 0.905792236328125
Precision: 0.9099177821027128
Recall: 0.9654838165879973
Epoch 0...... Average Loss for Epoch: 0.49673006041296597
F1: 0.9060837159267726
Accuracy: 0.850799560546875
Precision: 0.8360452337906342
Recall: 0.988929889298893
Epoch 0...... Average Loss for Epoch: 0.49362617529044717
F1: 0.933007481724441
Accuracy: 0.904632568359375
Precision: 0.9509264114665268
Recall: 0.9157513781929891
Epoch 0...... 

Epoch 0...... Average Loss for Epoch: 0.4214035627933649
F1: 0.9338768504612744
Accuracy: 0.90594482421875
Precision: 0.9479094076655052
Recall: 0.9202536997885835
Epoch 0...... Average Loss for Epoch: 0.4201624149367923
F1: 0.913929609012885
Accuracy: 0.881561279296875
Precision: 0.9680981018605526
Recall: 0.8655017431847776
Epoch 0...... Average Loss for Epoch: 0.41887084774251254
F1: 0.9402879300220706
Accuracy: 0.910003662109375
Precision: 0.910120727500784
Recall: 0.9725235602094241
Epoch 0...... Average Loss for Epoch: 0.4173723228066881
F1: 0.9425711384220017
Accuracy: 0.915435791015625
Precision: 0.9298711919852791
Recall: 0.9556227937468482
Epoch 0...... Average Loss for Epoch: 0.4159562044121601
F1: 0.9527666432497212
Accuracy: 0.93023681640625
Precision: 0.9342734419320853
Recall: 0.9720067453625633
Epoch 0...... Average Loss for Epoch: 0.4145983028302499
F1: 0.9357455284719798
Accuracy: 0.904510498046875
Precision: 0.9130034061310358
Recall: 0.9596495661696571
Epoch 0......

Epoch 0...... Average Loss for Epoch: 0.38137121808219265
F1: 0.941604645456776
Accuracy: 0.912841796875
Precision: 0.911920792079208
Recall: 0.973285992053428
Epoch 0...... Average Loss for Epoch: 0.38052016265930666
F1: 0.9448723060457893
Accuracy: 0.919830322265625
Precision: 0.9420847805163828
Recall: 0.9476763764943593
Epoch 0...... Average Loss for Epoch: 0.3797103092074394
F1: 0.9577990787999502
Accuracy: 0.93792724609375
Precision: 0.9440490797546012
Recall: 0.971955533097524
Epoch 0...... Average Loss for Epoch: 0.3789173085598429
F1: 0.9426959558595608
Accuracy: 0.915374755859375
Precision: 0.929689410613842
Recall: 0.9560715932430733
Epoch 0...... Average Loss for Epoch: 0.37813790878163106
F1: 0.9514474013514641
Accuracy: 0.928955078125
Precision: 0.9401533261891023
Recall: 0.9630161276703538
Epoch 0...... Average Loss for Epoch: 0.3774435000974427
F1: 0.9351519502273271
Accuracy: 0.909027099609375
Precision: 0.9662830426182342
Recall: 0.9059641728134878
Epoch 0...... Avera

Epoch 1...... Average Loss for Epoch: 0.278779068754779
F1: 0.9267326732673267
Accuracy: 0.8961181640625
Precision: 0.9498345466578425
Recall: 0.9047278840092456
Epoch 1...... Average Loss for Epoch: 0.27720830315037776
F1: 0.9441182682154172
Accuracy: 0.91925048828125
Precision: 0.9483643769358056
Recall: 0.9399100121946091
Epoch 1...... Average Loss for Epoch: 0.2764284312725067
F1: 0.951250564704916
Accuracy: 0.92755126953125
Precision: 0.9300140534029311
Recall: 0.9734795948388182
Epoch 1...... Average Loss for Epoch: 0.2759219223544711
F1: 0.9278042566276439
Accuracy: 0.899688720703125
Precision: 0.9681870272748109
Recall: 0.8906553091001096
Epoch 1...... Average Loss for Epoch: 0.27773004363883624
F1: 0.9487808281994538
Accuracy: 0.921600341796875
Precision: 0.9069217868577527
Recall: 0.9946908574056269
Epoch 1...... Average Loss for Epoch: 0.27732386407644855
F1: 0.933388733379986
Accuracy: 0.90704345703125
Precision: 0.9678897002131616
Recall: 0.901262722243338
Epoch 1...... Av

Epoch 1...... Average Loss for Epoch: 0.2686285568752151
F1: 0.9434979828173666
Accuracy: 0.917510986328125
Precision: 0.94049008168028
Recall: 0.9465251855890617
Epoch 1...... Average Loss for Epoch: 0.26885119314704625
F1: 0.9163785319444139
Accuracy: 0.884124755859375
Precision: 0.9669098852070456
Recall: 0.8708664713269151
Epoch 1...... Average Loss for Epoch: 0.2693344251790517
F1: 0.9061361350173242
Accuracy: 0.871856689453125
Precision: 0.9664775165705021
Recall: 0.8528867194075072
Epoch 1...... Average Loss for Epoch: 0.2687796495027012
F1: 0.952563025210084
Accuracy: 0.93109130859375
Precision: 0.9521629567408652
Recall: 0.9529634300126103
Epoch 1...... Average Loss for Epoch: 0.2692074228639472
F1: 0.9490032823633016
Accuracy: 0.9222412109375
Precision: 0.9083524904214559
Recall: 0.9934629567549447
Epoch 1...... Average Loss for Epoch: 0.2688638414885547
F1: 0.9497755930754433
Accuracy: 0.92828369140625
Precision: 0.9629469122426869
Recall: 0.9369597301286106
Epoch 1...... Av

Epoch 1...... Average Loss for Epoch: 0.2695023000240326
F1: 0.9474259298171883
Accuracy: 0.92364501953125
Precision: 0.9481431635614249
Recall: 0.9467097803720657
Epoch 1...... Average Loss for Epoch: 0.26928481459617615
F1: 0.9549776228679853
Accuracy: 0.933380126953125
Precision: 0.9398010960016238
Recall: 0.9706523561965453
Epoch 1...... Average Loss for Epoch: 0.2691822279183591
F1: 0.9371775417298938
Accuracy: 0.91156005859375
Precision: 0.9694577745885097
Recall: 0.9069777199681115
Epoch 1...... Average Loss for Epoch: 0.26890625919752975
F1: 0.9512076580098473
Accuracy: 0.929534912109375
Precision: 0.9545358157682684
Recall: 0.947902628032345
Epoch 1...... Average Loss for Epoch: 0.2690437791808959
F1: 0.9524655349898069
Accuracy: 0.928131103515625
Precision: 0.917162293488824
Recall: 0.9905953480560921
Epoch 1...... Average Loss for Epoch: 0.2691973376274109
F1: 0.9492965117449326
Accuracy: 0.923126220703125
Precision: 0.9114486703772418
Recall: 0.9904237893233652
Epoch 1.....

Epoch 1...... Average Loss for Epoch: 0.26976536529905654
F1: 0.9494843190907177
Accuracy: 0.9267578125
Precision: 0.9524111139261887
Recall: 0.9465754574450227
Epoch 1...... Average Loss for Epoch: 0.2695613174933439
F1: 0.9495232040686585
Accuracy: 0.92730712890625
Precision: 0.9540518673082655
Recall: 0.9450373307461931
Epoch 1...... Average Loss for Epoch: 0.27047664299607277
F1: 0.8704394655099528
Accuracy: 0.784881591796875
Precision: 0.7725107660185306
Recall: 0.9968006735424121
Epoch 1...... Average Loss for Epoch: 0.2704225853516187
F1: 0.9486132276744894
Accuracy: 0.922821044921875
Precision: 0.9136561117851971
Recall: 0.9863517282176962
Epoch 1...... Average Loss for Epoch: 0.2704046931246231
F1: 0.9548755609955121
Accuracy: 0.9324951171875
Precision: 0.9227615029767772
Recall: 0.989305490975187
Epoch 1...... Average Loss for Epoch: 0.270373233301299
F1: 0.9471961861667744
Accuracy: 0.92022705078125
Precision: 0.909038036524369
Recall: 0.9886981824315776
Epoch 1...... Averag

In [79]:
pre_test = model(x_for_nn_test)
_, predicted_test = torch.max(pre_test.data, 1)
print(f'F1: {f1_score(y_for_nn_test, predicted_test)}')
print(f'Accuracy: {accuracy_score(y_for_nn_test, predicted_test)}')
print(f'Precision: {precision_score(y_for_nn_test, predicted_test)}')
print(f'Recall: {recall_score(y_for_nn_test, predicted_test)}')

F1: 0.956988752704643
Accuracy: 0.9372249832977562
Precision: 0.9527979319304917
Recall: 0.961216602462774


In [80]:
df_test = dfs_parquet['Wednesday-28-02-2018']
#Transforming timestamp and category columns and also scaling data using minmax scalar
df_test['Timestamp'] = pd.to_datetime(df_test['Timestamp'])
df_test['Date'] = pd.to_datetime(df_test['Timestamp']).dt.date
df_test['TS_relative'] = (df_test['Timestamp'].astype(int) - 
                             pd.to_datetime(df_test['Date']).astype(int))/ 10**9
df_test = df_test.drop(['Timestamp'], axis = 1)
df_test = df_test.drop(['Date'], axis = 1)
df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
df_test.fillna(0, inplace=True)
df_test[cat_cols] = df_test[cat_cols].astype('category')
X_test = df_test.drop(['is_allowed'], axis = 1)
y_test = df_test['is_allowed'].astype(int)*1

In [81]:
stdsc = MinMaxScaler()
stdsc.fit(X_test)
X_test_scaled = stdsc.transform(X_test)
x_for_nn_test =  torch.from_numpy(X_test_scaled).float()
x_for_nn_test.shape
y_for_nn_test = torch.from_numpy(y_test.to_numpy())
y_for_nn_test.shape

torch.Size([613071])

In [82]:
pre_test = model(x_for_nn_test)
_, predicted_test = torch.max(pre_test.data, 1)
print(f'F1: {f1_score(y_for_nn_test, predicted_test)}')
print(f'Accuracy: {accuracy_score(y_for_nn_test, predicted_test)}')
print(f'Precision: {precision_score(y_for_nn_test, predicted_test)}')
print(f'Recall: {recall_score(y_for_nn_test, predicted_test)}')

F1: 0.7754044128302481
Accuracy: 0.6460768817967251
Precision: 0.8878006693656196
Recall: 0.6882690187431092
